## Анализ датафреймов

Задача проанализировать все характеристики, потенциально влияющие на успеваемость студентов из 3-х датафреймов.
Датафреймы содержат данные как об активности студентов на учебной платформе, так и часть харакетристик учебного материала. На успеваемость могут влиять как действия студентов, так и качество/сложность материалов. Соответственно анализ поделим на 2 части:

**а. Разобъем студентов на кластеры по успеваемости и посмотрим есть ли различия в кластерах по характеристкам. Основные характеристики по студентам**:

          1. perforamce ratio = количество правильных ответов/все ответы студента
          2. lectures ratio = количество посещенных лекций/ все действия студента на платформе (гипотеза - больше лекций, лучше подготовка студентов)
          3. diversity category ratio = количество уникального материала (лекции и вопросы)/все действия студента на платформе (гипотеза - индекс позволит выявить повторение студентами материала и соответственно оно должно положительно влиять на успеваемость)
          4. explanations usage ratio = количество использования пояснений к вопросам/количество ответов студента (гипотеза - использование пояснений улучшает подготовку студентов, позитивный эффект на учебу)
          5. среднее время на ответ (гипотеза - быстрее отвечают более подготовленные студенты)
          6. среднее время на событие на платформе (гипотеза - более подготовленные студенты больше времени проводят на платформе)
          
**б. Разобъем вопросы на кластеры по количеству правильных ответов и посмотрим на разницу в их характеристиках. Основные характеристика вопросов**:

           1. % правильных ответов = правильные ответы/все ответы
           2. Кол-во лекции, к которым относится вопрос (гипотеза - чем больше лекций, тем легче ответить на вопрос)
           3. Кол-во частей в вопросе (гипотеза -  чем меньше частей, тем легче ответить)
           4. Кол-во прослушенных лекций студентами, к которым относится вопрос (гипотеза - чем больше лекций прослушано, тем легче ответить на вопрос)         

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

Прочитаем файл "train" итеративно в связи с большим объемом и выгрузим необходимую информацию агрегированно:   

          по Id каждого студента (aggregated_students)
          по ID каждого вопроса (aggregated_questions)
          по ID каждой лекции (aggregated_lectures)

In [2]:
chunk_size = 10**6
train = pd.read_csv('data/train.csv', sep=',', chunksize = chunk_size)

In [3]:
aggregated_students = pd.DataFrame()
aggregated_questions = pd.DataFrame()
aggregated_lectures = pd.DataFrame()

chunk_counter = 0

print("Chunk aggregation started. Wait...")

for chunk in train:
    details = chunk[['user_id',
                     'content_type_id',
                     'answered_correctly',
                     'prior_question_elapsed_time',
                     'content_id',
                     'prior_question_had_explanation',
                     'timestamp'
                    ]]
    details = details.reindex(columns = details.columns.tolist() + ['count'])  
    details['count'] = 1
      

    #Получим уникальные ID всех студентов и посчитаем все события, в которых они участвовали на платформе вкл лекции и вопросы
    unique_ids = details.groupby('user_id').count()[['count']]
    
    #Получим уникальные ID всех вопросов и посчитаем, сколько раз отвечали на каждый вопрос и сколько из них правильно
    questions = details[details['content_type_id'] == 0]
    unique_qu_ids = questions.groupby('content_id').agg(no = ('count', 'count'),
                                                        no_correct = ('answered_correctly', 'sum')
                                                       )
    #Получим уникальные ID всех лекций и посчитаем, сколько раз их слушали                                                   
    lectures = details[details['content_type_id'] == 1]
    unique_lec_ids = lectures.groupby('content_id').agg(no = ('count', 'count'))
                                                      
    #Начнем агрегацию по студентам в датафрейме students_agg
                                                        
    #Соедним данные с ID студентов,кол-вом событий с кол-вом вопросов, на которые отвечал студент
    total = questions.groupby('user_id').count()[['content_type_id']]
    students_agg = pd.merge(unique_ids, total, how = 'left', on = ['user_id'])
    
    #Соедним данные с ID студентов,кол-вом событий,кол-вом вопросов с кол-вом правильных ответов
    answered = questions[questions['answered_correctly'] == 1].groupby('user_id').\
                         count()[['answered_correctly']]
    students_agg = pd.merge(students_agg, answered, how = 'left', on = ['user_id'])
    
    #Соедним данные с ID студентов,кол-вом событий,кол-вом вопросов, кол-вом правильных ответов с временем на вопрос
    time = details.groupby('user_id').mean()[['prior_question_elapsed_time']]
    students_agg = pd.merge(students_agg, time, how = 'left', on = ['user_id'])
    
    #Добавим кол-во уникального контента, чтобы понять были ли повоторения
    content = details.groupby('user_id')['content_id'].nunique()
    students_agg = pd.merge(students_agg, content, how = 'left', on = ['user_id'])
    
    #Добавим сколько раз студент пользовался пояснениями к вопросам
    expl = details[details['prior_question_had_explanation'] == True]
    explanation = expl.groupby('user_id')['prior_question_had_explanation'].count()
    students_agg = pd.merge(students_agg, explanation, how = 'left', on = ['user_id'])
    
    #Добавим суммарное время с начала первого события и до послднего события на платформе
    time2 = details.groupby('user_id').max()[['timestamp']]
    students_agg = pd.merge(students_agg, time2, how = 'left', on = ['user_id'])
    
    
    #Начнем собирать сводные датафреймы по итогам всех итераций
    aggregated_students = aggregated_students.append(students_agg)
    aggregated_questions = aggregated_questions.append(unique_qu_ids)
    aggregated_lectures = aggregated_lectures.append(unique_lec_ids)
    
    chunk_counter+=1
    print(f"Log: agg {chunk_counter} done.")

Chunk aggregation started. Wait...
Log: agg 1 done.
Log: agg 2 done.
Log: agg 3 done.
Log: agg 4 done.
Log: agg 5 done.
Log: agg 6 done.
Log: agg 7 done.
Log: agg 8 done.
Log: agg 9 done.
Log: agg 10 done.
Log: agg 11 done.
Log: agg 12 done.
Log: agg 13 done.
Log: agg 14 done.
Log: agg 15 done.
Log: agg 16 done.
Log: agg 17 done.
Log: agg 18 done.
Log: agg 19 done.
Log: agg 20 done.
Log: agg 21 done.
Log: agg 22 done.
Log: agg 23 done.
Log: agg 24 done.
Log: agg 25 done.
Log: agg 26 done.
Log: agg 27 done.
Log: agg 28 done.
Log: agg 29 done.
Log: agg 30 done.
Log: agg 31 done.
Log: agg 32 done.
Log: agg 33 done.
Log: agg 34 done.
Log: agg 35 done.
Log: agg 36 done.
Log: agg 37 done.
Log: agg 38 done.
Log: agg 39 done.
Log: agg 40 done.
Log: agg 41 done.
Log: agg 42 done.
Log: agg 43 done.
Log: agg 44 done.
Log: agg 45 done.
Log: agg 46 done.
Log: agg 47 done.
Log: agg 48 done.
Log: agg 49 done.
Log: agg 50 done.
Log: agg 51 done.
Log: agg 52 done.
Log: agg 53 done.
Log: agg 54 done.
Lo

In [4]:
#Удалим дубликаты, которые могли появиться из-за итерационного чтения файла train
aggregated_students.reset_index().drop_duplicates(subset='user_id',keep='first').set_index('user_id')
aggregated_questions.reset_index().drop_duplicates(subset='content_id',keep='first').set_index('content_id')
aggregated_lectures.reset_index().drop_duplicates(subset='content_id',keep='first').set_index('content_id')

#Заменим на 0 пустые значения NaN, которые образовались при слиянии датафреймов
aggregated_students.replace(to_replace=np.nan, value=0, inplace=True)
aggregated_questions.replace(to_replace=np.nan, value=0, inplace=True)
aggregated_lectures.replace(to_replace=np.nan, value=0, inplace=True)

## Рассчитаем основные индексы и дополнительные характеристики необходиыме для этих индексов
      
      0. lectures no = количество лекций, которые посетил студент
      1. perform_ratio = количество правильных ответов/все ответы студента
      2. lectures_ratio = количество посещенных лекций/ все действия студента на платформе (гипотеза - больше лекций, лучше подготовка студентов)
      3. content_ratio = количество уникального материала (лекции и вопросы)/все действия студента на платформе (гипотеза - индекс позволит выявить повторение студентами материала и соответственно оно должно положительно влиять на успеваемость)
      4. expl_ratio = количество использования пояснений к вопросам/количество ответов студента (гипотеза - использование пояснений улучшает подготовку студентов, позитивный эффект на учебу)
      5. avg_time_eventsсреднее время на событие на платформе (гипотеза - более подготовленные студенты больше времени проводят на платформе)


In [5]:
aggregated_students['lectures_no'] = aggregated_students['count']-aggregated_students['content_type_id']
aggregated_students['perform_ratio'] = round((aggregated_students['answered_correctly']/\
                                              aggregated_students['content_type_id'])*100,2)
aggregated_students['lectures_ratio'] = round((aggregated_students['lectures_no']/aggregated_students['count'])*100,2)
aggregated_students['content_ratio'] = round((aggregated_students['content_id']/aggregated_students['count'])*100,2)
aggregated_students['expl_ratio'] = round((aggregated_students['prior_question_had_explanation']/\
                                           aggregated_students['content_type_id'])*100,2)
aggregated_students['avg_time_events'] = round((aggregated_students['timestamp']/aggregated_students['count']),2)


In [6]:
# Добавим колонку с группой по успеваемости в зависимости от индекса успеваемости
def perf_ratio_group(ratios):
    if ratios > 0 and ratios <= 35:
        return 'Group 1 below 35%'
    elif ratios > 35 and ratios <= 50:
        return 'Group 2 35-50%'
    elif ratios > 50 and ratios <= 70:
        return 'Group 3 50-70%'
    else:
        return 'Group 4 above 70%'

aggregated_students['perf_group'] =aggregated_students['perform_ratio'].apply(perf_ratio_group)

In [7]:
#Переведем формат timestamp в количество дней, которые студент провел на платформе
aggregated_students['time_events'] = aggregated_students['timestamp'].apply(lambda x: datetime.fromtimestamp(x/1000)\
                                           - pd.to_datetime('1970-01-01 00:00:00.000')).dt.days

In [8]:
#Сведем все данные по 4-м группам студентов в одну таблицу
final = aggregated_students.groupby('perf_group').agg(no_students = ('count', 'count'),
                                             no_events = ('count', 'mean'),
                                             lect_ratio = ('lectures_ratio', 'mean'),
                                             content_ratio = ('content_ratio', 'mean'),
                                             expl_ratio = ('expl_ratio', 'mean'),
                                             avg_time_qu = ('prior_question_elapsed_time', 'mean'),
                                             avg_time_events = ('time_events', 'mean')
                                            )
display(final)                                     

,no_students,no_events,lect_ratio,content_ratio,expl_ratio,avg_time_qu,avg_time_events
perf_group,,,,,,,
Group 1 below 35%,56461,39.175112,0.297731,98.874596,17.112358,22142.440133,16.070172
Group 2 35-50%,89221,75.312157,0.743228,97.063649,40.764188,25139.761261,34.528553
Group 3 50-70%,180546,297.801596,1.265441,94.392535,68.204486,24198.719987,75.787805
Group 4 above 70%,67529,570.601564,0.940876,93.922704,73.852606,22868.167348,83.931496


## Выводы по группировке студентов по успеваемости

По активности на платформе в виде среднего кол-ва событий, в которых участвуют студенты **(no_events)**, самая большая активность у группы 4 - с лучшей подготовкой. Активность положительно влияет на успеваемость.

По среднему кол-ву лекций в расчете на все события на платформе, на которых были студенты **(lect_ratio)**, опять лидируют группы 3 и 4. Положительный эффект на успеваемость не столько очевиден, как в предыдущем случае, но имеет смысл включить и эту характеристику в анализ.

Средний индекс по разнообразию изучаемого контента **(сonten_ratio)** ниже всех также у 4-й группы, это значит, что у этой группы больше всего повторений материала и попыток сдачи вопросов. Здесь скорее всего есть положительный эффект на успеваемость.

Индекс использования пояснений **(expl_ratio)** намного выше у 4-й самой успевающей группы, очевидна положительная связь с успеваемостью.

Среднее время ответа на вопросы **(avg_time_qu)** самые низкие у группы 1 (худшая успеваемость) и группы 4 (лучшая успеваемость). Влияние этого фактора не столь очевидно, нужны дополнительные исследования. Возможно быстро отвечают, когда хорошо знают и также когда вообще не знают материал :)

Среднее время на событие на платформе **(avg_time_events)** выше у 4-й самой успевающей группы, что говорит о том, что чем дольше работает студент на платформе, тем потенциальнее выше его успеваемость. Ученье - труд :)

В основном, начальные гипотезы по указанным факторам визуально подтвердились, но статистический анализ взаимосвязей не проводился.

In [9]:
#Прочитаем файл с данными по лекциям и вопросам. Здесь объем небольшой, читаем обычно.
lectures = pd.read_csv('data/lectures.csv', sep=',')
questions = pd.read_csv('data/questions.csv', sep=',')

Начнем рассчитывать основные характеристики для анализа вопросов. Начнем с:

           1. perform_ratio  % правильных ответов = правильные ответы/все ответы
   
Для расчета остальных характеристик необходимо подготовить доп данные

           2. Кол-во лекций, к которым относится вопрос (гипотеза - чем больше лекций, тем легче ответить на вопрос)
           3. Кол-во частей в вопросе (гипотеза -  чем меньше частей, тем легче ответить)
           4. Кол-во прослушенных лекций студентами, к которым относится вопрос (гипотеза - чем больше лекций прослушано, тем легче ответить на вопрос

In [10]:
aggregated_questions['perform_ratio'] = round((aggregated_questions['no_correct']/\
                                              aggregated_questions['no'])*100,2)

In [11]:
# Объединим датафрейм с вопросам с агрегированным датафреймом, полученным из train
aggregated_questions = pd.merge(aggregated_questions, questions, how = 'left', left_on='content_id', right_on = 'question_id')

#Добавим в дата фрейм по вопросам группу по успеваемости. Критерии аналогичные кластерам студентов, поэтому та же функция
aggregated_questions['perf_group'] =aggregated_questions['perform_ratio'].apply(perf_ratio_group)

In [12]:
# Аналогично поступаем с лекциями, только не будем по ним анализировать успеваемость
aggregated_lectures = pd.merge(aggregated_lectures, lectures, how = 'left', left_on='content_id', right_on='lecture_id')

Для связи лекций и вопросов будем использовать теги, которые есть в обоих файлах. Создадим список уникальных тегов из файла с лекциями и суммируем частоту прослушивания каждой лекции по тегу.

In [13]:
tag_lectures = aggregated_lectures.groupby('tag').agg(no = ('no', 'sum'))

In [14]:
# Разобьем теги из файла по вопросам на отдельные столбцы, сохраним в отдельный временный файл
new = pd.DataFrame(aggregated_questions['tags'].str.split(' ').fillna('[]').tolist())    

# Разбивка была не совсем удачной, появились значения None и скобки. Избавимся от них, заменив на 3 - такой лекции нет
new.replace(to_replace=[None], value=3, inplace=True)
new.replace(to_replace='[', value=3, inplace=True)
new.replace(to_replace=']', value=3, inplace=True)
new

,0,1,2,3,4,5
0,51,131,162,38,3,3
1,131,36,81,3,3,3
2,131,101,162,92,3,3
3,131,149,162,29,3,3
4,131,5,162,38,3,3
...,...,...,...,...,...,...
1339425,8,3,3,3,3,3
1339426,14,3,3,3,3,3
1339427,8,3,3,3,3,3
1339428,125,3,3,3,3,3


In [15]:
# На основании полученных тегов из файла с лекциями подтянем частоту прослушивания каждой лекции. Если тега нет, то 0.
def get_freq(tag):
    try:
        tag1 = int(tag)
        value = tag_lectures.loc[tag1][0]
        return value
    except KeyError:
        return 0
    
new['freq0'] = new[0].apply(get_freq)
display('Added 1.')
new['freq1'] = new[1].apply(get_freq)
display('Added 2.')
new['freq2'] = new[2].apply(get_freq)
display('Added 3.')
new['freq3'] = new[3].apply(get_freq)
display('Added 4.')
new['freq4'] = new[4].apply(get_freq)
display('All done')
new

'Added 1.'

'Added 2.'

'Added 3.'

'Added 4.'

'All done'

,0,1,2,3,4,5,freq0,freq1,freq2,freq3,freq4
0,51,131,162,38,3,3,4449,0,0,0,0
1,131,36,81,3,3,3,0,3631,0,0,0
2,131,101,162,92,3,3,0,4871,0,0,0
3,131,149,162,29,3,3,0,3833,0,0,0
4,131,5,162,38,3,3,0,7173,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1339425,8,3,3,3,3,3,23565,0,0,0,0
1339426,14,3,3,3,3,3,11013,0,0,0,0
1339427,8,3,3,3,3,3,23565,0,0,0,0
1339428,125,3,3,3,3,3,14187,0,0,0,0


In [16]:
# Добавим файл с частотой прослушивания лекций в агрегированный файл по вопросам и подсчиатем суммы чатоты для каждого вопроса
aggregated_questions = pd.concat([aggregated_questions,new], axis = 1)
aggregated_questions['tot_fr'] = aggregated_questions[['freq0','freq1','freq2','freq3','freq4']].sum(axis=1)

In [17]:
# Посчитаем для каждого вопроса, сколько лекций нужно для его ответа
def get_lecture (tag):
    try:
        num = tag.split(' ')
        return len(num)
    except AttributeError:
        return 0
aggregated_questions['no lectures'] =aggregated_questions['tags'].apply(get_lecture)

In [18]:
#Сведем все характеристики по вопросам в кластеры
final_qu = aggregated_questions.groupby('perf_group').agg(no_quest = ('no', 'count'),
                                                         no_lect = ('no lectures', 'mean'),
                                                         no_parts = ('part', 'mean'),             
                                                         lect_freq = ('tot_fr', 'mean')
                                                         )
display(final_qu)   

,no_quest,no_lect,no_parts,lect_freq
perf_group,,,,
Group 1 below 35%,57621,1.884469,4.830166,23771.353395
Group 2 35-50%,141122,1.941979,4.636924,22715.543615
Group 3 50-70%,375185,2.055991,4.512417,21454.545400
Group 4 above 70%,765502,2.471673,4.061584,20876.631284


## Выводы по группировке вопросов по успешности ответов

Группа 4, которая включает вопросы с максимальным кол-вом правильных ответов, самая многочисленная (больше 50%) - колонка **(no_quest)** .

По кол-ву лекций, которые относятся к вопросам **(no_lect)**, лидирует опять 4-я группа. Это говорит о том, что чем больше лекции включают вопрос, то вероятность правильного ответа выше.

Интересная тендеция и по кол-ву частей **(no_part)**. Самый маленький показатель у 4-й группы, что возможно говорит о том, что меньшее кол-во частей в вопросе позволяет лучше отвечать на него.

Протовоположную гипотезе тенденцию показал анализ частоты прослушанных лекции по вопросу **(lect_freq)**. Получается, что по тем вопросам, по которым было прослушано больше лекций, студенты хуже отвечали. Думаю, этот вопрос требует дальнейшего анализа.

Все остальные гипотеза подтвердились и характеристики можно включать в анализ.